In [0]:
-- list all tables
SHOW TABLES;

-- describe table dim_customers
DESCRIBE TABLE dim_customers;

-- list unique customer countries
SELECT DISTINCT country
  FROM dim_customers;


-- list product categories, subcategories, and products
SELECT DISTINCT
  category,
  subcategory,
  product_name
  FROM dim_products
  ORDER BY category, subcategory, product_name;


-- calculate total sales
SELECT 
  SUM(sales_amount) AS total_sales
  FROM fact_sales;

-- calculate total customers
SELECT 
  COUNT(DISTINCT customer_key) AS total_customers
  FROM dim_customers;

-- calculate total products
SELECT
  COUNT(DISTINCT product_key) AS total_products
  FROM dim_products;

-- calculate total orders
SELECT
  COUNT(DISTINCT order_number) AS total_orders
  FROM fact_sales;


-- calculate sales by product, sort by highest sales
SELECT 
  p.product_name,
  SUM(f.sales_amount) AS total_sales
  FROM fact_sales f 
  JOIN dim_products p
    ON f.product_key = p.product_key
  GROUP BY p.product_name
  ORDER BY SUM(f.sales_amount) DESC;

-- calculate sales by category, sort by highest sales
SELECT 
  p.category,
  SUM(f.sales_amount) AS total_sales
  FROM fact_sales f 
  JOIN dim_products p
    ON f.product_key = p.product_key
  GROUP BY p.category
  ORDER BY SUM(f.sales_amount) DESC;

-- calculate sales over time (month), sort chronologically
SELECT 
  date_trunc('month', order_date) as month,
  SUM(sales_amount) AS total_sales
  FROM salesdb.fact_sales
  GROUP BY month
  ORDER BY month;


-- calculate total profit (assuming profit = sales_amount - cost * quantity)
SELECT 
  SUM(f.sales_amount - (p.cost * f.quantity)) AS total_profit
  FROM fact_sales f
  JOIN dim_products p
    ON f.product_key = p.product_key;


-- calculate total quantity
SELECT 
  SUM(quantity)AS total_quantity
  FROM fact_sales;

-- calculate avg sale per customer
SELECT 
  AVG(sales_amount) AS avg_sales
  FROM fact_sales;


-- calculate nr of orders by customer age group
SELECT 
  CASE 
    -- FLOOR is used to round down the calculated age to the nearest whole number
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) BETWEEN 0 AND 18 THEN '0-18'
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) BETWEEN 19 AND 30 THEN '19-30'
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) BETWEEN 31 AND 40 THEN '31-40'
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) BETWEEN 41 AND 50 THEN '41-50'
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) BETWEEN 51 AND 60 THEN '51-60'
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) BETWEEN 61 AND 70 THEN '61-70'
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) BETWEEN 71 AND 80 THEN '71-80'
    WHEN FLOOR(DATEDIFF(CURRENT_DATE(), c.birthdate) / 365.25) >= 81 THEN '81+'
    ELSE 'Unknown'
  END AS age_group,
  COUNT(DISTINCT f.order_number) AS total_orders
FROM workspace.salesdb.fact_sales f
JOIN workspace.salesdb.dim_customers c
  ON f.customer_key = c.customer_key
GROUP BY age_group
ORDER BY age_group;

-- calculate top 10 customers by sales
SELECT 
  c.first_name || ' ' || c.last_name AS customer,
  SUM(f.sales_amount) AS total_sales
  FROM fact_sales f
  JOIN dim_customers c
    ON f.customer_key = c.customer_key
  GROUP BY c.first_name || ' ' || c.last_name
  ORDER BY SUM(f.sales_amount) DESC
  LIMIT 10;

  
